In [1]:
from enum import Enum

# 七節　温水暖房

## 付録B　石油温水暖房機及び石油給湯温水暖房機

In [2]:
def get_kerosene_cons(hs_type, q_out, temp_sw, temp_rw, eff_rtd, q_rtd):
    
    def get_f_rtd(hs_type, temp_rw):
        if hs_type == 'kerConv':
            return 0.946
        elif hs_type == 'kerCondensing':
            temp_rw_rtd = 50.0
            if temp_rw_rtd >= 46.5:
                if temp_rw >= 46.5:
                    return 0.9768
                else:
                    return (-0.0023 * temp_rw + 1.014) / 0.907 * 0.9768
            else:
                if temp_rw >= 46.5:
                    return 0.907 / (-0.0023 * temp_rw_rtd + 1.014) * 0.9768
                else:
                    return (-0.0023 * temp_rw + 1.014) / (-0.0023 * temp_rw_rtd + 1.014) * 0.9768

    # heat loss from body, MJ/h (eq.2)
    q_body = {
        'kerConv' : 234 * 3600 * 10**(-6),
        'kerCondensing' : (5.3928 * temp_sw - 71.903) * 3600 * 10**(-6)
    }
    
    # (eq.5)
    f_rtd = get_f_rtd(hs_type, temp_rw)
    
    temp_sw_rtd = 70.0

    # (eq.4)
    q_body_rtd = {
        'kerConv' : 234.0 * 3600 * 10**(-6),
        'kerCondensing' : (5.3928 * temp_sw_rtd - 71.903) * 3600 * 10**(-6)
    }
    
    # (eq.3)
    heat_exchange_coeff = eff_rtd * f_rtd * (q_rtd * 3600 * 10**(-6) + q_body_rtd) / (q_rtd * 3600 * 10**(-6))

    # hourly kerosene consumption, MJ/h (eq.1)
    return (q_out + q_body) / heat_exchange_coeff if q_out > 0.0 else 0.0

In [3]:
def get_elec_cons(hs_type, r_ws, ker_cons):
    # pump power, W
    p_pmp = {
        'kerConv' : 90.0,
        'kerCondensing' : 70.0
    }[hs_type]
    # (eq.7)
    elec_pmp = p_pmp * r_ws * 10**(-3)
    r = 0.003
    # (eq.8)
    elec_fan = ker_cons * r * 10**3 / 3600
    # (eq.6)
    return elec_pmp + elec_fan       

In [4]:
def get_cons(hs_type, q_out, temp_sw, temp_rw, eff_rtd, q_rtd, r_ws):
    gas = 0.0
    ker = get_kerosene_cons(hs_type, q_out, temp_sw, temp_rw, eff_rtd, q_rtd)
    elec = get_elec_cons(hs_type, r_ws, ker)
    return elec, gas, ker

In [5]:
class HeatRadiator_MainRoom_Exist(Enum):
    Exist = 1,
    NotExist = 2

class HeatRadiator_OtherRoom_Exist(Enum):
    Exist = 1,
    NotExist = 2
    

In [6]:
# 付録H　温水暖房用熱源機の最大能力

# 暖冷房区画の床面積には、温水暖房により暖房される暖冷房区画のみを積算する。

# 主居室、その他居室の放熱器の設置有無　＊　運転方法

def get_q_max_hs(_region, _A_HCZ_i, _HeatRadiator_MainRoom_Exist, _HeatRadiator_OtherRoom_Exist, _HeatRadiator_MainRoom_Operation, _HeatRadiator_OtherRoom_Operation):
    
    _f_CT = 1.05
    
    # _q_rq_H
    if _HeatRadiator_MainRoom_Exist == HeatRadiator_MainRoom_Exist.Exist and _HeatRadiator_OtherRoom_Exist == HeatRadiator_OtherRoom_Exist.Exist:
        _HeatRadiator_Exist = 'BothRoom_Exist'
    elif _HeatRadiator_MainRoom_Exist == HeatRadiator_MainRoom_Exist.Exist and _HeatRadiator_OtherRoom_Exist == HeatRadiator_OtherRoom_Exist.NotExist:
        _HeatRadiator_Exist = 'MainRoom_Exist'
    elif _HeatRadiator_MainRoom_Exist == HeatRadiator_MainRoom_Exist.NotExist and _HeatRadiator_OtherRoom_Exist == HeatRadiator_OtherRoom_Exist.Exist:
        _HeatRadiator_Exist = 'OtherRoom_Exist'
    
    coef = {'Region1' : {'BothRoom_Exist' : 90.02, 'MainRoom_Exist' : 139.26, 'OtherRoom_Exist' : 62.28},
            'Region2' : {'BothRoom_Exist' : 77.81, 'MainRoom_Exist' : 120.65, 'OtherRoom_Exist' : 53.26},
            'Region3' : {'BothRoom_Exist' : 73.86, 'MainRoom_Exist' : 111.32, 'OtherRoom_Exist' : 83.81},
            'Region4' : {'BothRoom_Exist' : 77.74, 'MainRoom_Exist' : 118.98, 'OtherRoom_Exist' : 55.41},
            'Region5' : {'BothRoom_Exist' : 83.24, 'MainRoom_Exist' : 126.56, 'OtherRoom_Exist' : 59.43},
            'Region6' : {'BothRoom_Exist' : 69.76, 'MainRoom_Exist' : 106.48, 'OtherRoom_Exist' : 49.93},
            'Region7' : {'BothRoom_Exist' : 74.66, 'MainRoom_Exist' : 112.91, 'OtherRoom_Exist' : 53.48},
            'Region8' : {'BothRoom_Exist' : None,  'MainRoom_Exist' : None,   'OtherRoom_Exist' : None}
    }
    
    _q_rq_H = coef[_region][_HeatRadiator_Exist]
    
    # _f_CI
      # 主たる居室の運転方法、その他の居室の運転方法
    _f_CI = {'Continuous'   : {'Continuous' : 1.0 , 'Intermittent' : 1.0 , 'NotExist' : 1.0 },
             'Intermittent' : {'Continuous' : 2.25, 'Intermittent' : 2.25, 'NotExist' : 3.03},
             'NotExist'     : {'Continuous' : 1.0 , 'Intermittent' : 1.62}
    }[_HeatRadiator_MainRoom_Operation][_HeatRadiator_OtherRoom_Operation]
    
    print(_q_rq_H , sum(_A_HCZ_i) , _f_CT , _f_CI, sep=", ")
    
    return _q_rq_H * sum(_A_HCZ_i) * _f_CT * _f_CI
    
   

#### Example

In [7]:
get_q_max_hs(_region = 'Region2', _A_HCZ_i = (29.81,0), _HeatRadiator_MainRoom_Exist = HeatRadiator_MainRoom_Exist.NotExist, _HeatRadiator_OtherRoom_Exist = HeatRadiator_OtherRoom_Exist.Exist, _HeatRadiator_MainRoom_Operation = 'Intermittent', _HeatRadiator_OtherRoom_Operation = 'Intermittent')

53.26, 29.81, 1.05, 2.25


3750.8954175000003

In [8]:
# 式（9）
def get_Q_max_H_hs(_q_rtd_hs):
    return _q_rtd_hs * 3600 / 10**(6)


#### Exapmle

In [9]:
_q_rtd_hs = get_q_max_hs(_region = 'Region2', _A_HCZ_i = (29.81,0, 0, 0, 0), _HeatRadiator_MainRoom_Exist = HeatRadiator_MainRoom_Exist.NotExist, _HeatRadiator_OtherRoom_Exist = HeatRadiator_OtherRoom_Exist.Exist, _HeatRadiator_MainRoom_Operation = 'NotExist', _HeatRadiator_OtherRoom_Operation = 'Intermittent')

_Q_max_H_hs = get_Q_max_H_hs(_q_rtd_hs)
_Q_max_H_hs
    

53.26, 29.81, 1.05, 1.62


9.722320922160002